In [1]:
%pip install datasets sentencepiece


  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached aiohttp-3.11.11-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached huggingface_hub-0.27.1-py3-none-any.whl.metadata (13 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-24.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached frozenlist-1.5.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached multidict-6.1.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.0 kB)
  Using cached propcache-0.2.1-cp312-cp312-manylinux_2_17_x86_

In [3]:
import pandas as pd
from transformers import AutoTokenizer


/home/olana/Desktop/KAIM 3/Week 5/ethiomart-ner/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
%pip install amseg

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for amseg: filename=amseg-2.3-py3-none-any.whl size=9826 sha256=cd4cfba7598a9ac4f23aaf01bb27352ba9876c849dd211faee19bdac08d6dd34
  Stored in directory: /home/olana/.cache/pip/wheels/62/07/fe/e03d5a7b2df7b8978bdce9d9b2cd5a48597de57f8270d3d560
Successfully built amseg
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install pdf2image

  Using cached pillow-11.1.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
Using cached pillow-11.1.0-cp312-cp312-manylinux_2_28_x86_64.whl (4.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


if you want to play with GPT token playground to see how it tokenize words and sentences [GPT token playground](https:/gpt-tokenizer.dev/)



In [11]:
with open('../data/labeled/labeled_telegram_product_price_location.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

    # Process lines as needed
data = [line.strip().split('\t') for line in lines]  # Adjust the split based on your delimiter
df = pd.DataFrame(data)

In [12]:
df

,0
0,እንኳን B-PRODUCT
1,ለጥምቀት I-PRODUCT
2,በዓል I-PRODUCT
3,በሰላም I-PRODUCT
4,አደረሳችሁ! I-PRODUCT
...,...
171970,join O
171971,ያድርጉ፣ O
171972,ቤተሰብ O
171973,ይሁኑ O


In [13]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

## Usinng different Amh tokenizers from HuggingFace



In [15]:
# Initialize the tokenizer
model_name = "xlm-roberta-base"  # Change to any approprate model from Hugging Face if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [16]:
# Function to tokenize and align labels
def tokenize_and_align_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = tokenizer.tokenize(word)  # Tokenize the word
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels

In [17]:
# Tokenize and align labels
aligned_tokens, aligned_labels = tokenize_and_align_labels(tokenizer, tokens, labels)

In [18]:
# Display the first 10 results
for token, label in zip(aligned_tokens[:20], aligned_labels[:20]):
    print(f"{token:20} {label}")


▁እንኳን                B-PRODUCT
▁ለ                   I-PRODUCT
ጥም                   O
ቀት                   O
▁በዓል                 I-PRODUCT
▁በሰላም                I-PRODUCT
▁አ                   I-PRODUCT
ደረ                   O
ሳ                    O
ችሁ                   O
!                    O
▁Wish                O
ing                  O
▁you                 O
▁a                   O
▁happy               O
▁Epi                 O
phan                 O
y                    O
,                    O


### Using betterTokenizer which google's sentence piece





link for the tokenizer: [link](https://github.com/uhh-lt/amharicprocessor)

In [19]:
from amseg.amharicSegmenter import AmharicSegmenter


In [20]:
# Initialize the Amharic segmenter
sent_punct = []
word_punct = []
segmenter = AmharicSegmenter(sent_punct, word_punct)

In [21]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

In [22]:
# Function to align tokens with their respective labels
def align_tokens_with_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = tokenizer.amharic_tokenizer(word)  # Tokenize the word
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels

In [23]:
# Example usage with different variable names
new_tokens, new_labels = align_tokens_with_labels(segmenter, tokens, labels)

In [24]:
# Output the first 20 aligned tokens and labels
for token, label in zip(new_tokens[:20], new_labels[:20]):
    print(f"{token}: {label}")


እንኳን: B-PRODUCT
ለጥምቀት: I-PRODUCT
በዓል: I-PRODUCT
በሰላም: I-PRODUCT
አደረሳችሁ: I-PRODUCT
!: O
Wishing: O
you: O
a: O
happy: O
Epiphany,: O
መልካም: O
በዓል: O
y'all: O
Kemei: B-PRODUCT
®: I-PRODUCT
Rechargeable: I-PRODUCT
Hair: I-PRODUCT
Clipper: I-PRODUCT
የፀጉር: O


In [25]:
# Saving the final tokens and labels to a CSV file
output_df = pd.DataFrame({'Token': new_tokens, 'Label': new_labels})
output_df.to_csv('../data/processed/final_tokens_labels.csv', index=False)